# Data Processsing

In [1]:
train_dir = '/home/ec2-user/SageMaker/train/'
test_dir = '/home/ec2-user/SageMaker/test/'
files_dir = '/home/ec2-user/SageMaker/'

## Labels

In [2]:
import json

with open(files_dir + '/label_num_to_disease_map.json', 'r') as f:
    labels_info = json.load(f)

NUM_LABELS = len(labels_info)
print(labels_info)

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


## Load Data with Pytorch

In [3]:
!pip install efficientnet_pytorch torch torchvision;

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
!pip install timm;

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
import pandas as pd
import numpy as np

import os
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

import timm
from efficientnet_pytorch import EfficientNet

from PIL import Image
from sklearn.model_selection import train_test_split

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


## Test Dataset

In [7]:
class CassavaDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.image_labels = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_labels.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.image_labels.iloc[index, 1]
        
        # ResNet Transform
        image_resnet = self.transform['resnet'](image)
        
        # VT Transform
        image_vt = self.transform['vt'](image)
        
        # Efficient Net Transform
        image_efficient = self.transform['efficient'](image)

        return image_resnet, image_vt, image_efficient, label

In [8]:
df = pd.read_csv(files_dir + '/train.csv')

# Split the data into train, validation, and test sets with a ratio of 80:10:10
train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

In [9]:
# Transforms for resnet
resnet_weights = models.ResNet50_Weights.DEFAULT
resnet_preprocess = resnet_weights.transforms()
resnet_transforms = transforms.Compose([
    resnet_preprocess
])

In [10]:
# transforms for VT
vt_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [11]:
# transforms for efficient
efficient_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [12]:
transforms = {'resnet':resnet_transforms, 'vt': vt_transforms, 'efficient': efficient_transforms}
test_dataset = CassavaDataset(test_df, train_dir, transform=transforms)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Saved Models

## ResNet-50

In [13]:
resnet50 = models.resnet50()
resnet50.fc = torch.nn.Sequential(
    torch.nn.Linear(resnet50.fc.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, NUM_LABELS)
)

resnet_state_dict = torch.load(files_dir + 'saved_models/resnet50_5_epochs_512_batch.pth')
resnet50.load_state_dict(resnet_state_dict)
# resnet50.to(device)
resnet50.eval();

## Vision Transformer

In [14]:
vt = timm.create_model("vit_base_patch16_224", pretrained=False)
vt.head = torch.nn.Sequential(
    torch.nn.Linear(vt.head.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, NUM_LABELS)
)

vt_state_dict = torch.load(files_dir + 'saved_models/vit_10_epochs_16_batch.pth')
vt.load_state_dict(vt_state_dict)
# vt.to(device)
vt.eval();

## EfficientNet

In [15]:
effNet = EfficientNet.from_pretrained('efficientnet-b0')

effNet._fc = torch.nn.Sequential(
    torch.nn.Linear(effNet._fc.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, NUM_LABELS)
)

effNet_state_dict = torch.load(files_dir + 'saved_models/effNet_b0_2_ep ochs_32_batch.pth')
effNet.load_state_dict(effNet_state_dict)
# effNet.to(device)
effNet.eval();

Loaded pretrained weights for efficientnet-b0


# Inference using Ensemble

In [16]:
# Calculate the test accuracy
test_predictions = np.array([])
true_labels = np.array([])

total = 0
correct = 0

for resnet_input, vt_input, eff_input, labels in test_loader:
    # labels = labels.to(device)
    
    # ResNet
    # resnet_input = resnet_input.to(device)
    resnet_predictions = resnet50(resnet_input)
    
    # VT
    # vt_input = vt_input.to(device)
    vt_predictions = vt(vt_input)
    
    # EfficientNet
    # eff_input = eff_input.to(device)
    effNet_predictions = effNet(eff_input)
    
    predictions = vt_predictions + resnet_predictions + effNet_predictions
    
    _, predicted = torch.max(predictions, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
    true_labels = np.append(true_labels, labels.cpu().numpy())
    test_predictions = np.concatenate((test_predictions, predicted.detach().cpu().numpy()))
    
print("Test Accuracy: ", correct/total)

Test Accuracy:  0.861214953271028


In [17]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

f1 = f1_score(true_labels, test_predictions, average='weighted')
recall = recall_score(true_labels, test_predictions, average='weighted')
precision = precision_score(true_labels, test_predictions, average='weighted')
accuracy = accuracy_score(true_labels, test_predictions)

# Print the accuracy, F1 score, recall, and precision
print("Test Accuracy: ", accuracy)
print("F1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)

Test Accuracy:  0.861214953271028
F1 Score:  0.857112755450272
Recall:  0.861214953271028
Precision:  0.8584637925393761
